<a href="https://colab.research.google.com/github/TheOnlylight/C/blob/master/BrainExplanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Brain Pulse analysis from fMRI aspect

*this project and its thory are inspired by the SPM fMRI*

By reading the source code of the 1st level analysis, it is not hard to find that the regressor X in $Y = X \times B + \epsilon$ is generated using the *spm_Volterra.m* this function is in charge of 
```
% Generalized convolution of inputs (U) with basis set (bf)
% FORMAT [X,Xname,Fc] = spm_Volterra(U,bf,V)
% U          -  input structure array (see spm_get_ons.m)
% bf         -  Basis functions (see spm_get_bf.m)
% V          -  [1 or 2] order of Volterra expansion [default = 1]
%
% X          -  Design Matrix
% Xname      -  names of regressors [columns] in X
% Fc(i).i    -  indices pertaining to input i (and interactions)
% Fc(i).name -  names pertaining to input i   (and interactions)
% Fc(i).p    -  grouping of regressors per parameter
%__________________________________________________________________________
%
% For first order expansions spm_Volterra simply convolves the causes (e.g.
% stick functions) in U.u by the basis functions in bf to create a design
% matrix X.  For second order expansions new entries appear in X, Xname and
% Fc that correspond to the interaction among the original causes. The
% basis functions for these effects are two dimensional and are used to
% assemble the second order kernel in spm_graph.m. Second order effects are
% computed for only the first column of U.u.
```
this function requries bf --> bf         -  Basis functions (see spm_get_bf.m)

the function *spm_get_bf.m* is in charge of generating the original pulse function.
```
% Fill in basis function structure
% FORMAT [xBF] = spm_get_bf(xBF)
%
% xBF.dt      - time bin length {seconds}
% xBF.name    - description of basis functions specified
%               'hrf'
%               'hrf (with time derivative)'
%               'hrf (with time and dispersion derivatives)'
%               'Fourier set'
%               'Fourier set (Hanning)'
%               'Gamma functions'
%               'Finite Impulse Response'
%               (any other specification will default to 'hrf')
% xBF.length  - window length (seconds)
% xBF.order   - order
% xBF.T       - microtime resolution (for 'hrf*')
%
% xBF.bf      - array of basis functions
%__________________________________________________________________________
%
% spm_get_bf prompts for basis functions to model event or epoch-related
% responses.  The basis functions returned are unitary and orthonormal
% when defined as a function of peri-stimulus time in time-bins.
% It is at this point that the distinction between event and epoch-related 
% responses enters.
```

In [ ]:
!pip install smop
# Generated with SMOP  0.41
from libsmop import *
# spm_get_bf.m

    
@function
def spm_get_bf(xBF=None,*args,**kwargs):
    varargin = spm_get_bf.varargin
    nargin = spm_get_bf.nargin

    # Fill in basis function structure
# FORMAT [xBF] = spm_get_bf(xBF)
    
    # xBF.dt      - time bin length {seconds}
# xBF.name    - description of basis functions specified
#               'hrf'
#               'hrf (with time derivative)'
#               'hrf (with time and dispersion derivatives)'
#               'Fourier set'
#               'Fourier set (Hanning)'
#               'Gamma functions'
#               'Finite Impulse Response'
#               (any other specification will default to 'hrf')
# xBF.length  - window length (seconds)
# xBF.order   - order
# xBF.T       - microtime resolution (for 'hrf*')
    
    # xBF.bf      - array of basis functions
#__________________________________________________________________________
    
    # spm_get_bf prompts for basis functions to model event or epoch-related
# responses.  The basis functions returned are unitary and orthonormal
# when defined as a function of peri-stimulus time in time-bins.
# It is at this point that the distinction between event and epoch-related 
# responses enters.
#__________________________________________________________________________
# Copyright (C) 1999-2011 Wellcome Trust Centre for Neuroimaging
    
    # Karl Friston
# $Id: spm_get_bf.m 4473 2011-09-08 18:07:45Z guillaume $
    
    
    #-Length of time bin
#--------------------------------------------------------------------------
    if logical_not(nargin):
        str='time bin for basis functions {secs}'
# spm_get_bf.m:37
        xBF.dt = copy(spm_input(str,'+1','r',1 / 16,1))
# spm_get_bf.m:38
    
    dt=xBF.dt
# spm_get_bf.m:40
    #-Assemble basis functions
#==========================================================================
    
    #-Model event-related responses
#--------------------------------------------------------------------------
    if logical_not(isfield(xBF,'name')):
        spm_input('Hemodynamic Basis functions...',1,'d')
        Ctype=cellarray(['hrf','hrf (with time derivative)','hrf (with time and dispersion derivatives)','Fourier set','Fourier set (Hanning)','Gamma functions','Finite Impulse Response'])
# spm_get_bf.m:50
        str='Select basis set'
# spm_get_bf.m:58
        Sel=spm_input(str,2,'m',Ctype)
# spm_get_bf.m:59
        xBF.name = copy(Ctype[Sel])
# spm_get_bf.m:60
    
    
    #-Get order and length parameters
#--------------------------------------------------------------------------
    if cellarray(['Fourier set','Fourier set (Hanning)','Gamma functions','Finite Impulse Response']) == xBF.name:
        #----------------------------------------------------------------------
        try:
            l=xBF.length
# spm_get_bf.m:70
        finally:
            pass
        try:
            h=xBF.order
# spm_get_bf.m:74
        finally:
            pass
    
    #-Create basis functions
#==========================================================================
    if cellarray(['Fourier set','Fourier set (Hanning)']) == xBF.name:
        #----------------------------------------------------------------------
        pst=concat([arange(0,l,dt)]).T
# spm_get_bf.m:87
        pst=pst / max(pst)
# spm_get_bf.m:88
        #----------------------------------------------------------------------
        if strcmp(xBF.name,'Fourier set (Hanning)'):
            g=(1 - cos(dot(dot(2,pi),pst))) / 2
# spm_get_bf.m:93
        else:
            g=ones(size(pst))
# spm_get_bf.m:95
        #-Zeroth and higher Fourier terms
    #----------------------------------------------------------------------
        bf=copy(g)
# spm_get_bf.m:100
        for i in arange(1,h).reshape(-1):
            bf=concat([bf,multiply(g,sin(dot(dot(dot(i,2),pi),pst)))])
# spm_get_bf.m:102
            bf=concat([bf,multiply(g,cos(dot(dot(dot(i,2),pi),pst)))])
# spm_get_bf.m:103
    else:
        if cellarray(['Gamma functions']) == xBF.name:
            #----------------------------------------------------------------------
            pst=concat([arange(0,l,dt)]).T
# spm_get_bf.m:108
            bf=spm_gamma_bf(pst,h)
# spm_get_bf.m:109
        else:
            if cellarray(['Finite Impulse Response']) == xBF.name:
                #----------------------------------------------------------------------
                bin=l / h
# spm_get_bf.m:113
                bf=kron(eye(h),ones(round(bin / dt),1))
# spm_get_bf.m:114
            else:
                if cellarray(['NONE']) == xBF.name:
                    #----------------------------------------------------------------------
                    bf=1
# spm_get_bf.m:118
                else:
                    #-Microtime resolution
    #----------------------------------------------------------------------
                    try:
                        fMRI_T=xBF.T
# spm_get_bf.m:124
                    finally:
                        pass
                    #-Canonical hemodynamic response function
    #----------------------------------------------------------------------
                    bf,p=spm_hrf(dt,[],fMRI_T,nargout=2)
# spm_get_bf.m:131
                    #----------------------------------------------------------------------
                    if strfind(xBF.name,'time'):
                        dp=1
# spm_get_bf.m:137
                        p[6]=p(6) + dp
# spm_get_bf.m:138
                        D=(bf(arange(),1) - spm_hrf(dt,p,fMRI_T)) / dp
# spm_get_bf.m:139
                        bf=concat([bf,ravel(D)])
# spm_get_bf.m:140
                        p[6]=p(6) - dp
# spm_get_bf.m:141
                        #------------------------------------------------------------------
                        if strfind(xBF.name,'dispersion'):
                            dp=0.01
# spm_get_bf.m:147
                            p[3]=p(3) + dp
# spm_get_bf.m:148
                            D=(bf(arange(),1) - spm_hrf(dt,p,fMRI_T)) / dp
# spm_get_bf.m:149
                            bf=concat([bf,ravel(D)])
# spm_get_bf.m:150
                    #-Length and order
    #----------------------------------------------------------------------
                    xBF.length = copy(dot(size(bf,1),dt))
# spm_get_bf.m:156
                    xBF.order = copy(size(bf,2))
# spm_get_bf.m:157
    
    
    
    #-Orthogonalise and fill in basis function structure
#--------------------------------------------------------------------------
    xBF.bf = copy(spm_orth(bf))
# spm_get_bf.m:164
    #==========================================================================
#- S U B - F U N C T I O N S
#==========================================================================
    
    
@function
def spm_gamma_bf(u=None,h=None,*args,**kwargs):
    varargin = spm_gamma_bf.varargin
    nargin = spm_gamma_bf.nargin

    # Return basis functions (Gamma functions) used for Volterra expansion
# FORMAT bf = spm_gamma_bf(u,h);
# u   - times {seconds}
# h   - order
# bf  - basis functions (mixture of Gammas)
#__________________________________________________________________________
    u=ravel(u)
# spm_get_bf.m:178
    bf=[]
# spm_get_bf.m:179
    for i in arange(2,(1 + h)).reshape(-1):
        m=2 ** i
# spm_get_bf.m:181
        s=sqrt(m)
# spm_get_bf.m:182
        bf=concat([bf,spm_Gpdf(u,(m / s) ** 2,m / s ** 2)])
# spm_get_bf.m:183
    